In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import pandas as pd 
import numpy as np
import plotly.io as pio
import glob
import os
from scipy import stats 
offline.init_notebook_mode(connected=True)
from IPython.display import display
from math import ceil, floor




In [ ]:
arr=!echo $LOGDIR
FOLDER=arr[0]

In [ ]:
#Place / at the end of path

#Use paramter0_run/ for IMT=100 and use parameter1_run/ for IMT=10
FOLDER="/zfs/dicelab/DigitalPlant/results/July_23_16_48_08_node0229_Partition/parameter0_run/"
# pd.set_option('display.max_columns', None)


In [ ]:
#FOLDER="./"
df = [{"name":"4 Partitions", "df":None, "color":'#F56600'}, {"name":"8 Partitions", "df":None, "color":'#522D80'}, {"name":"16 Partitions", "df":None, "color":'#546223'}, {"name":"32 Partitions", "df":None, "color":'#005EB8'}] 
params = ["connection_strings_4.list/", "connection_strings_8.list/", "connection_strings_16.list/", "connection_strings_32.list/"]

for i in range(4):
    #For IMT=10: it seems like trial3 was the best run, all of the graphs work for trial 4 as well.
    
    #Wajdi, dropping messages is broken for trial0 and trial1.
    path = FOLDER+params[i]+'trial7/'
    all_files = glob.glob(path + "*.csv")
    li = []
    
    for filename in all_files:
        frame = pd.read_csv(filename, index_col=None, header=0)
        li.append(frame)

    frame = pd.concat(li, axis=0, ignore_index=True)
    
#     frame = pd.read_csv(FOLDER+params[i]+'log999_latencies.csv', na_filter= False)
    
    frame['Receive Time'] = pd.to_datetime(frame['Receive Time'], unit='ms')
    frame['Send Time'] = pd.to_datetime(frame['Send Time'], unit='ms')
    frame['Latency'] = (frame['Receive Time'] - frame['Send Time']).dt.total_seconds() * 1000
    df[i]["df"] = frame

# i = 0

# for param in params:
#     if os.path.isdir(FOLDER + param):
#         all_files = []

#         partitions = FOLDER + param

#         for partition in os.listdir(partitions):
#             all_files += glob.glob(partitions + partition + "/" + "*.csv")

#         li = []

#         for filename in all_files:
#             frame = pd.read_csv(filename, index_col=None, header=0)
#             li.append(frame)

#         frame = pd.concat(li, axis=0, ignore_index=True)

#         frame['Receive Time'] = pd.to_datetime(frame['Receive Time'], unit='ms')
#         frame['Send Time'] = pd.to_datetime(frame['Send Time'], unit='ms')
#         frame['Latency'] = (frame['Receive Time'] - frame['Send Time']).dt.total_seconds() * 1000

#         df[i]["df"] = frame

#         i += 1
    
    
    
display(df)

In [ ]:
!cat {FOLDER}parameter*

In [ ]:
for frame in df:
    print(frame["df"].groupby(['Status']).count())

# df.groupby(['Status']).count()

In [ ]:
#get rid of the connection message
PERCENT_TO_DROP = 0.05

for frame in df:
    num_messages = frame["df"].shape[0]
    messages_to_drop = 45
    print("Original dataframe contained {} messages, with each device sending {}\nDropping {}%, or {} messages from the beginning".format(len(frame["df"]), (num_messages/1000), PERCENT_TO_DROP,messages_to_drop))
    frame["df"].drop(frame["df"][frame["df"]['Message ID'] < messages_to_drop].index, inplace=True) 
    
    
    origSend = frame["df"].at[messages_to_drop, "Send Time"]
    frame["df"]["Relative Send Time"] = frame["df"]["Send Time"] - origSend
    frame["df"]['Relative Send Time'] = frame["df"]['Relative Send Time'].apply(lambda x: float(x.total_seconds())) 
    frame["df"].count()
    print(frame["df"])


In [ ]:
fig_avg_lat = go.Figure()

# for frame in df:
# for i in range(4):
     
frame = df[0]
bins = np.linspace(frame["df"]["Relative Send Time"].min(), frame["df"]["Relative Send Time"].max(), 285)
groups = frame["df"].groupby(pd.cut(frame["df"]["Relative Send Time"], bins))
fig_avg_lat.add_trace(go.Scatter(y=groups.mean().Latency, name=frame["name"], line=dict(color=frame["color"], width=2,
                              dash='dot')))

frame = df[1]
bins = np.linspace(frame["df"]["Relative Send Time"].min(), frame["df"]["Relative Send Time"].max(), 285)
groups = frame["df"].groupby(pd.cut(frame["df"]["Relative Send Time"], bins))
fig_avg_lat.add_trace(go.Scatter(y=groups.mean().Latency, name=frame["name"], line=dict(color=frame["color"], width=2,
                              dash='dash')))

frame = df[2]
bins = np.linspace(frame["df"]["Relative Send Time"].min(), frame["df"]["Relative Send Time"].max(), 285)
groups = frame["df"].groupby(pd.cut(frame["df"]["Relative Send Time"], bins))
fig_avg_lat.add_trace(go.Scatter(y=groups.mean().Latency, name=frame["name"], line=dict(color=frame["color"], width=2,
                              dash='dashdot')))

frame = df[3]
bins = np.linspace(frame["df"]["Relative Send Time"].min(), frame["df"]["Relative Send Time"].max(), 285)
groups = frame["df"].groupby(pd.cut(frame["df"]["Relative Send Time"], bins))
fig_avg_lat.add_trace(go.Scatter(y=groups.mean().Latency, name=frame["name"], line=dict(color=frame["color"], width=2)))

fig_avg_lat.update_layout(
    xaxis_title = "Time (s)",
    yaxis_title = "Average Latency (ms)",
    yaxis_gridcolor = '#ccc',
    plot_bgcolor = '#fff',
    font=dict(
        family="Computer Modern",
        size=30,
        color="#000"
    ),
    legend=dict(x=-.1, y=1.2, orientation="h")
) 
    
fig_avg_lat.show()

In [ ]:
for frame in df:
    print(frame["df"]['Latency'].describe())


In [ ]:
fig_box = go.Figure()

for frame in df:
    fig_box.add_trace(go.Box(y=frame["df"]["Latency"], quartilemethod="inclusive", name=frame["name"], marker_color=frame["color"]))
    
fig_box.update_layout(
    title = "Latency vs Intermessage Time",
    xaxis_title = "Number of Partitions",
    yaxis_title = "Latency (ms)"
)    

fig_box.show()

In [ ]:
# fig_cdf = go.Figure()

# for frame in df:
#     totalMessages = len(frame["df"])
#     freq = frame["df"].groupby(['Latency']).size()
#     rel = freq / totalMessages # Divide that by the total messages to get the relative count
#     cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
#     fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, mode='lines', name=frame["name"], marker_color=frame["color"]))


# fig_cdf.update_layout(
#     xaxis_title = "Latency (ms)",
#     yaxis_title = "Probability",
#     xaxis_range = [25, 200],
#     yaxis_gridcolor = '#ccc',
#     plot_bgcolor = '#fff',
#     font=dict(
#         family="Computer Modern",
#         size=30,
#         color="#000"
#     )
# ) 


# fig_cdf.show()


fig_cdf = go.Figure()

frame = df[0]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name=frame["name"], line=dict(color=frame["color"], width=4,
                              dash='dot')))


frame = df[1]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name=frame["name"], line=dict(color=frame["color"], width=4, dash='dash')))


frame = df[2]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name=frame["name"], line=dict(color=frame["color"], width=4)))


frame = df[3]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name=frame["name"], line=dict(color=frame["color"], width=4, dash='dashdot')))

fig_cdf.update_layout(
    legend=dict(x=-0.1, y=1.2, orientation="h",
        font = dict(
            family = 'Computer Modern',
            size = 24,
            color = '#000'
        )
    ),
    
    plot_bgcolor = '#fff',
    
    xaxis = dict(
        range = [25, 200],
        showgrid = True,
        zeroline = True,
        showline = True,
        gridcolor = '#fff',
        title = 'Latency (ms)',
        titlefont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        ),
        showticklabels = True,
        tickfont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        )
    ),
    
    yaxis = dict(
        showgrid = True,
        zeroline = True,
        showline = True,
        gridcolor = '#ccc',
        title = 'Probability',
        titlefont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        ),
        showticklabels = True,
        tickfont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        )
    )
)

fig_cdf.show()

In [ ]:
fig_mes_lat = go.Figure()

# frame = df[0]
# frame = df[1]
# frame = df[2]
# frame = df[3]
# fig_mes_lat.add_trace(go.Scatter(x=frame["df"]["Relative Send Time"], y=frame["df"]["Latency"], mode='lines', name=frame["name"], marker_color=frame["color"]))

for frame in df:
    fig_mes_lat.add_trace(go.Scatter(x=frame["df"]["Relative Send Time"], y=frame["df"]["Latency"], mode='lines', name=frame["name"], marker_color=frame["color"]))

fig_mes_lat.update_layout(
    title = "Message Latency (ms) vs Relative Send Time (s)",
    xaxis_title = "Relative Send Time (s)",
    yaxis_title = "Message Latency (ms)"
) 
    
fig_mes_lat.show()



# fig_ts = px.scatter(df, x="Receive Time", y="Latency", color="deviceNum", title='Time Received vs Latency (ms) Colored by Device', render_mode='webgl')
# fig_ts.update_traces(line=dict(width=1))
# fig_ts.update_layout(
#     showlegend=False,
#     yaxis_title="Latency (ms)")
# fig_ts.show()